# Multi-h5 files Processing: PFLOTRAN Variables Mapping and E4D Forward Run

This note book is the third part of the repository. Make sure you have finished **PartA_E4D_Install_Kernel_Customize** and select **your own kernel** for this notebook. By running this notebook, users can **map the PFLOTRAN Simulation Variables to E4D mesh to generate bulk conductivity** and **run E4D Forward Mode**.<br>
The users should provide the **mesh interplolation matrix _map.bin** and **E4D Mesh Files** or you can generate these files from **PartB_Mesh_Interpolation**. Also, the **PFOTRAN Simulation Result Files** are needed.<br>
Brief introduction of the notebooks and tips will be included before or after the relevant cell in this notebook.<br> 
For more detailed information to run the E4D Forward Mode, please read **T.C. Johnson's** (E4D Developer) tutorial: https://e4d.pnnl.gov/Documents/E4D_User_Guide.pdf. I also upload one copy in this folder.<br>
Please see the plot below the workflow of **PartB** and **PartC**. **PartB** includes from **Step 1: E4D Mesh Generation to Step 2: Mesh Interpolation Matrix**. **PartC** includes from **Step 3: PFLOTRAN Simulation Time Extraction to Step 6: E4D Forward Simulation**.

# 1. Pre-processing

## 1.1 Modules Import

To run the notebook, you need the packages below:<br>
   1. imp
   2. os
   3. IPyhon
   4. numpy
   5. h5py
   6. ipywidgets<br>
   
This step will check if all the required libraries installed or not. If not, the commands below will install it automatically.

<font color = red>Tips: The commands can only detect the libraries installed via **pip** and **conda**, which include most of the libraries installed. I will try other commands to include all libraries and update.<br>
The Python version for the kernel is Python3.6 and there are some updated packages in Python3.7, which means sometimes when you run '!pip install package_name --user', you can't install that because it exists in Python3.7 but you can't import it in Python3.7. The best way is to switch your Python to 3.7 in your **'kernel-helper.sh'**. However, Python3.7 will cause some problems, not very stable, so you can install the packages in the terminal after activating your kernel: <br>
    Open a terminal, and activate the selected kernel. Then use the command: **pip install package_name**<br>
    
After that, run **import package_name** again.<br><font>

In [1]:
# Import packages
import os
import shutil
import imp
import numpy as np
import math
import h5py
import ipywidgets as widgets

# Widgets style definition
style = {'description_width': '150px'}

## 1.2 Read Input Files

PartB needs some files as input. Among them, **Fortran Script Files** and **PFLOTRAN Simulation Result Files** should be provided by users. See the table below for what information these files can provide.<br>

| File |Function|Comments|
|:----:|:------:|:-------:|
|Fortran Script|Map PFLOTRAN Variable to E4D Mesh|Files with extension '.cfg'|
|E4D Mesh Files|E4D Mesh element, node, face and translation information|Files with extension '.1.ele', '.1.node', '.1.face' and 'trn'|
|PFLOTRAN Simulation Result File|PFLOTRAN Simulated Variables|Time Steps, Tracer, Saturation, Porosity, Temperature|
|Baseline Conductivity|Rock Conductivity|Files with extension '.sig'|

In this step, users can just check the essential files are avialable or not and do not select which one to use.

In [2]:
# input_file_shared folder path
input_source_path = '/global/u1/v/vthw28/1/' 

# read input file name
f_file_name = open(input_source_path+'parameter.txt')
f_file_name_lines = f_file_name.readlines()
prefix = input_source_path+f_file_name_lines[0].split()[1]
print('The prefix of all the E4D mesh files are the same: ','\x1b[0;30;42m' +prefix+ '\x1b[0m')
mesh_ele_name = prefix+'.1.ele'
mesh_node_name = prefix+'.1.node'
mesh_face_name = prefix+'.1.face'
mesh_trn_name = prefix+'.trn'

mapfile = input_source_path+f_file_name_lines[1].split()[1]
print('The Mesh Interpolation Matrix: ','\x1b[0;30;42m' +mapfile+ '\x1b[0m')

petro_file_name = input_source_path+f_file_name_lines[2].split()[1]
print('The petrophysical parameter file is ', '\x1b[0;30;42m'+petro_file_name+'\x1b[0m')

bc_file_name = input_source_path+f_file_name_lines[3].split()[1]
print ('The Baseline Conductivity File: ','\x1b[0;30;42m' +bc_file_name+ '\x1b[0m')

porosity_data_file = input_source_path+f_file_name_lines[4].split()[1]
print ('The Porosity Data File: ','\x1b[0;30;42m' +porosity_data_file+ '\x1b[0m')

f_file_name.close()

The prefix of all the E4D mesh files are the same:  /global/u1/v/vthw28/1/E4D_1
The Mesh Interpolation Matrix:  /global/u1/v/vthw28/1/E4D_1_map.bin
The petrophysical parameter file is  /global/u1/v/vthw28/1/wax_smit.txt
The Baseline Conductivity File:  /global/u1/v/vthw28/1/E4D_1.sig
The Porosity Data File:  /global/u1/v/vthw28/1/pflotran_porosity-000.h5


## Computational Survery File

This file is with the trasferred electrode location and the measurement are from 82800.srv file.

In [4]:
comp_srv_file_name = input_source_path+'E4D_comp.srv'

## 1.3 Fortran Script Compilation and Import

Compile and import the fortran scripts -- *test_interp2.f90* and *mapit.f90*.<br>
The command will check if you have compiled them before. If you have compiled it before, just import them. If not, the command will compile them first and import them.

In [3]:
import sys
sys.path.append(input_source_path)
import mapit

Compile 'test_interp2.f90' and 'mapit.f90' on NERSC:<br>
(1) Swap the programming environment from 'intel' to 'gnu';<br>
(2) Compile them in the terminal and check the 'python --version' and 'gcc --version' are the same in terminal and cell.<br>
(3) After running 'python -m numpy.f2py -c test_interp2.f90 -m test_interp2', a file named 'test_interp2.cpython-36m-x86_64-linux-gnu.so' will be generated and rename it as 'test_interp2.so'.<br>
There are some bugs on NERSC after last update, so check the compiler version.

## PFLOTRAN Simulation Result Files

In [5]:
# PFLOTRAN Simulation Results File
path = os.getcwd() # The path of current folder

pf_output_files = [f for f in os.listdir(path) if f.endswith('.h5')]
pf_output_l = len(pf_output_files)

if pf_output_l == 0:
    print('\x1b[0;30;41m' + 'PFLOTRAN Simulation File' + '\x1b[0m', 'is not available, please upload a PFLOTRAN Simulation Result file.')
else:
    print ('\x1b[0;30;42m' + 'PFLOTRAN Simulation Result File' + '\x1b[0m', 'has been found, please check:')
    print (pf_output_files)

PFLOTRAN Simulation Result File has been found, please check:
['pflotranR87-000.h5']


In [6]:
# Read the PFLOTRAN Simulation Result file to extract the simulation time and write them into 'prefix'_time.txt
for i in range(pf_output_l):
    pf_output_name_l = len(pf_output_files[i])
    pf_output_prefix = pf_output_files[i][0:(pf_output_name_l-3)]
    sim_time_file = pf_output_prefix+'_time.txt'

    f2 = open(sim_time_file,'w')

    f1 = h5py.File(pf_output_files[i],'r')
    for ky in f1.keys():
      if(ky[0:5]=='Time:'):
        f2.write(ky+'\n')
    
    f1.close()
    f2.close()

    # Write the simulation step file: First column - simulation step time; Second column - baseline conductivity file name;
    # Third column - petrophysical parameter file name
    sim_time = open(sim_time_file)
    sim_time_line = sim_time.readlines()
    sim_time_l = len(sim_time_line)

    f = open(pf_output_prefix + '_sim.txt','w+')
    f.write('pf_time'+' '+'bc_file'+' '+'petro_file')
    f.write('\n')

    for i in range(sim_time_l):
        sim_time_one_step = sim_time_line[i].split()
        f.write(str(sim_time_one_step[1]) + ' ' + bc_file_name + ' ' + petro_file_name)
        f.write('\n')
    
    f.close()

    simt_f = pf_output_prefix+'_sim.txt'

    print('Write simulation time, baseline conductivity file, and petrophysical parameter file into', '\x1b[0;30;42m'+simt_f+'\x1b[0m')

Write simulation time, baseline conductivity file, and petrophysical parameter file into pflotranR87-000_sim.txt


In [7]:
# Get PFLOTRAN Simulation time step
sim_time_files = [f for f in os.listdir(path) if f.endswith('_time.txt') and f.startswith('pflotran')]
sim_time_file_l = len(sim_time_files)
sim_time_step = []

for i in range(sim_time_file_l):
    f = open(sim_time_files[i])
    sim_time_lines = f.readlines()
    sime_time_line_l = len(sim_time_lines)
    
    for j in range(sime_time_line_l):
        sim_time_step.append(sim_time_lines[j].split()[1])
        
sim_time_step_float = [float(i) for i in sim_time_step]
sim_time_step_int = [int(i) for i in sim_time_step_float]
sim_time_step_int.sort()

In [8]:
obs_path = '/global/cscratch1/sd/vthw28/PartE_Forward_Mode/Observation_File'
obs_step = []

obs_files = [f for f in os.listdir(obs_path) if f.endswith('.srv')]
obs_files_l = len(obs_files)
for i in range(obs_files_l):
    obs_name_l = len(obs_files[i])
    obs_step.append(obs_files[i][0:(obs_name_l-4)])

obs_time_step_int = [int(i) for i in obs_step]
obs_time_step_int.sort()
obs_time_step_int.remove(871200)
obs_time_step_int_h = [c/3600 for c in obs_time_step_int]

In [9]:
obs_time_step_int_h_np = np.asarray(obs_time_step_int_h)
obs_time_step_int_h_np_734 = obs_time_step_int_h_np[obs_time_step_int_h_np < 734]
obs_time_step_int_h_np_734_l = len(obs_time_step_int_h_np_734)
obs_srv_name = []
sig_show_name = []

for j in range(obs_time_step_int_h_np_734_l):
    n = [abs(i-obs_time_step_int_h_np_734[j]) for i in sim_time_step_int]
    idx = n.index(min(n))
    print('For observation time step:','\x1b[0;30;42m'+str(int(obs_time_step_int_h_np_734[j]*3600))+'.srv'+'\x1b[0m','the PFLOTRAN Simulation time is:','\x1b[0;30;42m'+str(sim_time_step_int[idx])+' h'+'\x1b[0m')
    obs_srv_name.append(str(int(obs_time_step_int_h_np_734[j]*3600))+'.srv')
    sig_show_name.append(str(sim_time_step_int[idx]))
    
sig_show_name_float = [int(i) for i in sig_show_name]

For observation time step: 82800.srv the PFLOTRAN Simulation time is: 23 h
For observation time step: 439200.srv the PFLOTRAN Simulation time is: 122 h
For observation time step: 543600.srv the PFLOTRAN Simulation time is: 151 h
For observation time step: 615600.srv the PFLOTRAN Simulation time is: 171 h
For observation time step: 694800.srv the PFLOTRAN Simulation time is: 193 h
For observation time step: 795600.srv the PFLOTRAN Simulation time is: 221 h
For observation time step: 972000.srv the PFLOTRAN Simulation time is: 270 h
For observation time step: 1047600.srv the PFLOTRAN Simulation time is: 291 h
For observation time step: 1137600.srv the PFLOTRAN Simulation time is: 316 h
For observation time step: 1209600.srv the PFLOTRAN Simulation time is: 336 h
For observation time step: 1314000.srv the PFLOTRAN Simulation time is: 365 h
For observation time step: 1339200.srv the PFLOTRAN Simulation time is: 372 h
For observation time step: 1393200.srv the PFLOTRAN Simulation time is: 3

# 3 PFLOTRAN State Variables Extraction

In [10]:
porosity_file_open = h5py.File(porosity_data_file,'r')

key_time = []

for ky in porosity_file_open.keys():
    if (ky[0:5] == 'Time:'):
        key_time.append(ky)

grep = porosity_file_open[key_time[0]]
porosity = np.copy(grep.get('Porosity'),order='F') 

x,y,z = np.shape(porosity)
temp = np.ones(shape = (x,y,z))*25

porosity_file_open.close()

In [12]:
for i in range(pf_output_l):
    pf_output_file_name = pf_output_files[i]
    print('Modify the file: ','\x1b[0;30;42m'+pf_output_file_name+'\x1b[0m')
    f1 = h5py.File(pf_output_file_name,'r+')
    
    z_coord = np.copy(f1['Coordinates'].get('Z [m]'),order='F')

    pf_sat = 'Liquid_Saturation'
    pf_tracer = 'Total_Tracer [M]'
    pf_poro = 'Porosity'
    pf_tem = 'Temperature'

    for ky in f1.keys():
        if(ky[0:5]=='Time:'):
            #getthe current time
            t_cur = f1.get(ky)
            print(ky)
            pft = float(t_cur.name.split()[1])
            print('New step: ',str(pft))
            #open and loop over the simulation time file to see if 
            #we need to build the conductivity file for this simulation time
            sim_f = input_source_path+'pflotran_time.txt'
            f2=open(sim_f,'r')
            #f2.readline()
            for line in f2.readlines():
                if len(line.split())==4:
                    time = float(line.split()[1])
                    water_level = float(line.split()[3])
                
                    if(np.abs(pft-time)<0.001):
                        print ("Mapping time: ",time)
                    
                        # extract the pflotran state variables for this time
                    
                        for pvar in t_cur:
                            if(pvar == pf_sat):
                                #extract saturation for this time step
                                print ("Extracting variable: ",pvar," for: ",ky)
                                sat =  np.copy(t_cur.get(pvar),order='F')
                            elif(pvar == pf_tracer):
                                #extract fluid cond for this time step
                                print ("Extracting variable: ",pvar," for: ",ky)
                                fcond =  np.copy(t_cur.get(pvar),order='F')
                                
                        index_coord = np.argmax(z_coord > float(water_level))
                        index = index_coord-1 
                        
                        sat_below_table = sat[:,:,0:index]
                        fcond_below_table = fcond[:,:,0:index]
                        
                        sat_inter = sat[:,:,index]
                        fcond_inter = fcond[:,:,index]
                        
                        print(water_level)
                        print(z_coord[index])
                        
                        fcond_inter[sat_inter == 0] = (float(water_level)-z_coord[index])*2
                        sat_inter[sat_inter == 0] = (float(water_level)-z_coord[index])*2
                        print(round((float(water_level)-z_coord[index])*2,3))
                        
                        fcond[:,:,index] = fcond_inter
                        sat[:,:,index] = sat_inter
                        print(index)
                        print(np.shape(sat_below_table))
                        
                        fcond_below_table[sat_below_table == 0] = 1
                        fcond[:,:,0:index] = fcond_below_table
                        
                        sat_below_table[sat_below_table == 0] = 1
                        sat[:,:,0:index] = sat_below_table
                        
                        porosity[porosity < 0] = 1
                        fcond[fcond > 1] = 1
                        
                        print(ky)
                    
                        grp_2 = f1[ky]
                        
                        del grp_2['Total_Tracer [M]']
                        grp_2.create_dataset("Total_Tracer [M]", data = fcond)
                        
                        del grp_2['Liquid_Saturation']
                        grp_2.create_dataset("Liquid_Saturation", data = sat)
                        
                        
                        if pf_poro in grp_2.keys():
                            del grp_2['Porosity']
                            print('Delete porosity')
                        
                        grp_2.create_dataset("Porosity", data = porosity)
                        print('Add porosity')
                        
                        if pf_tem in grp_2.keys():
                            del grp_2['Temperature']
                            print('Delete temperature')
                            
                        grp_2.create_dataset("Temperature", data = temp)
                        print('Add temperature')
                                              
            f2.close
            
    f1.close()

Modify the file:  pflotranR87-000.h5
Time:  0.00000E+00 h
New step:  0.0
Mapping time:  0.0
Extracting variable:  Liquid_Saturation  for:  Time:  0.00000E+00 h
Extracting variable:  Total_Tracer [M]  for:  Time:  0.00000E+00 h
105.0793
105.0
0.159
30
(200, 200, 30)
Time:  0.00000E+00 h
Delete porosity
Add porosity
Delete temperature
Add temperature
Time:  1.22000E+02 h
New step:  122.0
Mapping time:  122.0
Extracting variable:  Liquid_Saturation  for:  Time:  1.22000E+02 h
Extracting variable:  Total_Tracer [M]  for:  Time:  1.22000E+02 h
105.73806
105.5
0.476
31
(200, 200, 31)
Time:  1.22000E+02 h
Delete porosity
Add porosity
Delete temperature
Add temperature
Time:  1.51000E+02 h
New step:  151.0
Mapping time:  151.0
Extracting variable:  Liquid_Saturation  for:  Time:  1.51000E+02 h
Extracting variable:  Total_Tracer [M]  for:  Time:  1.51000E+02 h
105.67291
105.5
0.346
31
(200, 200, 31)
Time:  1.51000E+02 h
Delete porosity
Add porosity
Delete temperature
Add temperature
Time:  1.71

In [13]:
f4 = h5py.File(pf_output_file_name,'r')
for ky in f4.keys():
    if(ky == "Coordinates"):
        coords = f4.get(ky)
        for k in coords.keys():
            if(k[0]=="X"):
                xnods=np.copy(coords.get(k),order='F')
                nx_nods = xnods.shape[0]
                nx_cells = nx_nods-1
            if(k[0]=="Y"):
                ynods=np.copy(coords.get(k),order='F')
                ny_nods = ynods.shape[0]
                ny_cells = ny_nods -1
            if(k[0]=="Z"):
                znods=np.copy(coords.get(k),order='F')
                nz_nods = znods.shape[0]
                nz_cells = nz_nods - 1
f4.close()
nv=nx_cells*ny_cells*nz_cells

# Input value of the conductivity of the surface and ground water.
print('Please input the conductivity of Surface Water and Ground Water (S/m)')

SW_cond_input = widgets.FloatText(value=0.012,description='Surface water Cond.(S/m):',disabled=False,style=style)
GW_cond_input = widgets.FloatText(value=0.040,description='Ground Water Cond.(S/m):',disabled=False,style=style)

display(SW_cond_input,GW_cond_input)

Please input the conductivity of Surface Water and Ground Water (S/m)


FloatText(value=0.012, description='Surface water Cond.(S/m):', style=DescriptionStyle(description_width='150p…

FloatText(value=0.04, description='Ground Water Cond.(S/m):', style=DescriptionStyle(description_width='150px'…

# 4 Conductivity Mapping at Simulation Step

Now, we get every files for the conductivity files ready. Call the module 'mapit' to map the bulk conductivity files at every simulation time.

In [14]:
sw_cond = SW_cond_input.value
gw_cond = GW_cond_input.value

for i in range(pf_output_l):
    pf_output_file_name = pf_output_files[i]
    print('Mapping variables from the file: ', '\x1b[0;30;42m'+pf_output_file_name+'\x1b[0m')
    pf_output_name_l = len(pf_output_files[i])
    pf_output_prefix = pf_output_files[i][0:(pf_output_name_l-3)]
    sim_f = pf_output_prefix+'_sim.txt'

    f1 = h5py.File(pf_output_file_name,'r')
    for ky in f1.keys():
      if(ky[0:5]=='Time:'):
        #getthe current time
        t_cur = f1.get(ky)
        pft = float(t_cur.name.split()[1])
        #open and loop over the simulation time file to see if 
        #we need to build the conductivity file for this simulation time
        f2=open(sim_f,'r')
        f2.readline()
        for line in f2.readlines():
          if len(line.split())==3:
            time = float(line.split()[0])
            if(np.abs(pft-time)<0.001) and time in sig_show_name_float:
              print ("Mapping time: ",time)
              sigfile = line.split()[1]
              petfile = line.split()[2]
    
              # extract the pflotran state variables for this time
          
              vcount = 0
              for pvar in t_cur:
                if(pvar == pf_sat):
                  #extract saturation for this time step
                  print ("Extracting variable: ",pvar," for: ",ky)
                  print('1')
                  vcount=vcount+1
                  sat =  np.copy(t_cur.get(pvar),order='F')
                elif(pvar == pf_tracer):
                  #extract fluid cond for this time step
                  print ("Extracting variable: ",pvar," for: ",ky)
                  print('2')
                  vcount=vcount+1
                  fcond =  np.copy(t_cur.get(pvar),order='F')
                  #transform river water fraction to fluid cond
                  #NOTE: this bit of code will be application specific
                  #If pflotran is simulating fluid conductivity then
                  #these lines can be commented out
                  for i in range(fcond.shape[0]):
                    fcond[i] = gw_cond - (gw_cond-sw_cond)*fcond[i]
                  print ('max and min fcond: ',np.max(fcond), np.min(fcond))
                  print('3')
                  print
                
            
                if(pvar == pf_poro):
                  #extract saturation for this time step
                  print ("Extracting variable: ",pvar," for: ",ky)
                  print('4')
                  vcount=vcount+1
                  porosity =  np.copy(t_cur.get(pvar),order='F')
                
                if(pvar == pf_tem):
                  #extract saturation for this time step
                  print ("Extracting variable: ",pvar," for: ",ky)
                  print('5')
                  vcount=vcount+1
                  temperature =  np.copy(t_cur.get(pvar),order='F')
          
              #check to make sure we've extracted the correct number of 
              #of pflotran variables. This part of the code will change
              #depending on how many variables are extract (1 to 4). 
          
              if(vcount != 4):
                print ("Did not find all of specified pflotran states")
                print ("in file ",pf_inp_f," for time ",time)
                exit()
          
              #reshape the arrays extracted from the h5 file in fortran order
              #in prep to send to map_waxsmit
          
              fcr = np.reshape(fcond,(nv,1),order='F')
              satr = np.reshape(sat,(nv,1),order='F')
              porosity = np.reshape(porosity,(nv,1),order='F')
              temperature = np.reshape(temperature,(nv,1),order='F')
          
              #send everything to map_waxsmit for interpolation
              mapit.map_waxsmit(fcr,satr,porosity,temperature,sigfile,petfile,mapfile,time)
                
                
        f2.close

    f1.close()

Mapping variables from the file:  pflotranR87-000.h5
Mapping time:  122.0
Extracting variable:  Liquid_Saturation  for:  Time:  1.22000E+02 h
1
Extracting variable:  Porosity  for:  Time:  1.22000E+02 h
4
Extracting variable:  Temperature  for:  Time:  1.22000E+02 h
5
Extracting variable:  Total_Tracer [M]  for:  Time:  1.22000E+02 h
2
max and min fcond:  0.04 0.012
3
Mapping time:  151.0
Extracting variable:  Liquid_Saturation  for:  Time:  1.51000E+02 h
1
Extracting variable:  Porosity  for:  Time:  1.51000E+02 h
4
Extracting variable:  Temperature  for:  Time:  1.51000E+02 h
5
Extracting variable:  Total_Tracer [M]  for:  Time:  1.51000E+02 h
2
max and min fcond:  0.04 0.012
3
Mapping time:  171.0
Extracting variable:  Liquid_Saturation  for:  Time:  1.71000E+02 h
1
Extracting variable:  Porosity  for:  Time:  1.71000E+02 h
4
Extracting variable:  Temperature  for:  Time:  1.71000E+02 h
5
Extracting variable:  Total_Tracer [M]  for:  Time:  1.71000E+02 h
2
max and min fcond:  0.04 0

In [14]:
sig_show_name_l = len(sig_show_name)
prefix_list = []

if len(pf_output_file_name) == 17:
    real_name = pf_output_file_name[8:10]
    
if len(pf_output_file_name) == 18:
    real_name = pf_output_file_name[8:11]
    
if len(pf_output_file_name) == 19:
    real_name = pf_output_file_name[8:12]
    
for i in range(sig_show_name_l):
    name = sig_show_name[i]+'.'
    prefixed = [filename for filename in os.listdir(path) if filename.startswith(name)]
    prefix_list.append(prefixed)
    prefixed_l = len(prefixed)
        
    if prefixed_l != 0:
        sig_name = prefixed[0]
        new_sig_name = 'e4d'+'-'+real_name+'-'+prefixed[0]
        os.rename(sig_name, new_sig_name)
        print('Mapping variables in file: ', '\x1b[0;30;42m'+new_sig_name+'\x1b[0m')

Mapping variables in file:  e4d-R87-23.0000000.sig
Mapping variables in file:  e4d-R87-122.000000.sig
Mapping variables in file:  e4d-R87-151.000000.sig
Mapping variables in file:  e4d-R87-171.000000.sig
Mapping variables in file:  e4d-R87-193.000000.sig
Mapping variables in file:  e4d-R87-221.000000.sig
Mapping variables in file:  e4d-R87-270.000000.sig
Mapping variables in file:  e4d-R87-291.000000.sig
Mapping variables in file:  e4d-R87-316.000000.sig
Mapping variables in file:  e4d-R87-336.000000.sig
Mapping variables in file:  e4d-R87-365.000000.sig
Mapping variables in file:  e4d-R87-372.000000.sig
Mapping variables in file:  e4d-R87-387.000000.sig
Mapping variables in file:  e4d-R87-408.000000.sig
Mapping variables in file:  e4d-R87-436.000000.sig
Mapping variables in file:  e4d-R87-574.000000.sig
Mapping variables in file:  e4d-R87-581.000000.sig
Mapping variables in file:  e4d-R87-588.000000.sig
Mapping variables in file:  e4d-R87-595.000000.sig
Mapping variables in file:  e4d

In [15]:
sig_show_name_l = len(sig_show_name)
sig_show_file = []

for i in range(sig_show_name_l):
    a = ''
    b = ''
    a = 'e4d-'+real_name+'-'+sig_show_name[i]+'.'
    b = [f for f in os.listdir(path) if f.startswith(a)]
    sig_show_file.append(b[0])
    
map_sh_name = 'map_e4d_sig_'+real_name+'.sh'

f = open(map_sh_name, "w+")

f.write('#!/bin/bash \n')
f.write('export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/global/project/projectdirs/m1800/E4D_20190818/e4d_dev/third_party/netcdf/liblib/.libs')
f.write('\n \n')

f.write('/global/project/projectdirs/m1800/E4D_20190818/e4d_dev/bin/bx -f E4D_1.1 e4d-'+ real_name+'-23.0000000.sig E4D_1_'+real_name+'.exo 23 h')
f.write('\n')

for j in range(sig_show_name_l-1):
    f.write('/global/project/projectdirs/m1800/E4D_20190818/e4d_dev/bin/bx -af E4D_1.1 ' + sig_show_file[j+1]+' ' + 'E4D_1_'+real_name+'.exo'+' '+sig_show_name[j+1]+' h')
    f.write('\n')

f.close()

In [16]:
# bash map_e4d_sig_R1.sh

In [17]:
real_name

'R87'

# 5 Run E4D Mode 2

In [18]:
f = open('e4d_forward.out',"w+")

f.write('1 \n')
f.write('e4d_forward_'+real_name+'.dpd')
f.write('\n')
f.write('0 \n')
f.write('0 \n')
f.write('0')

f.close()

In [19]:
f = open('sigs.lst',"w+")

f.write(str(len(prefix_list)))
f.write('\n')

for i in range(len(prefix_list)):
    f.write('e4d'+'-'+real_name+'-'+prefix_list[i][0])
    f.write(' ')
    a = prefix_list[i][0].split('.')
    b = int(a[0])*3600
    c = str(b)
    f.write(real_name+'_'+c)
    f.write('\n')

f.close()

In [20]:
f = open('e4d.inp',"w+")

f.write('2 \n')
f.write("'"+mesh_node_name+"'")
f.write('\n')
f.write("'"+comp_srv_file_name+"'")
f.write('\n')
f.write('sigs.lst \n')
f.write('e4d_forward.out \n')
f.write('0 \n')
f.write('0 \n')

f.close()